In [4]:
#Loading packages
#pip install tensorflow
#pip install keras
#pip install NLTK
import tensorflow
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Embedding
import pandas as pd
from keras.utils.np_utils import to_categorical
import os
from keras.layers import Activation, Dense
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [5]:
os.chdir("D:/VINODKUMAR/NLP CLASSIFICATION PROJECT/SHASTER-2/SHASTER")

In [6]:
#Reading the data
actual_data=pd.read_excel("Data-for-Classification-20180320.xlsx")
#data = pd.read_csv(open('traindata.csv','r'), encoding='utf-8', engine='c')
actual_data.columns

Index(['Summary (Input', 'DATA (Input)', 'categories (Output)',
       'sub_categories (Output)', 'previous_appointment (Output)'],
      dtype='object')

In [7]:
#Changing the column names
col_names=["summary","data","categories","sub_categories","appointment"]
actual_data.columns=col_names

In [8]:
actual_data.head()

,summary,data,categories,sub_categories,appointment
0,Pt aware that he needs ROV for refill,Caller Call Other Patient is returning nurse c...,PRESCRIPTION,REFILL,No
1,Mom wants to know if the Focalin needs some do...,Caller Other school teacher is reporting pt is...,ASK_A_DOCTOR,MEDICATION RELATED,No
2,pt called to discuss nortryptiline. she says s...,Home Prescription Other Prescription pt called...,ASK_A_DOCTOR,MEDICATION RELATED,No
3,FYI Nortryptline medication.,Home Prescription Other Prescription pt called...,MISCELLANEOUS,OTHERS,No
4,Letter of patient establishment request,Caller Other Requesting a letter stating that ...,MISCELLANEOUS,SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.),No


In [9]:
#Converting datatype of summary feature
actual_data.summary =actual_data.summary.astype('str')
#Converting datatype of data feature
actual_data.data =actual_data.data.astype('str')
#Converting datatype of category feature
actual_data.categories=actual_data.categories.astype("category")
#Converting datatype of sub_categories feature
actual_data.sub_categories=actual_data.sub_categories.astype("category")
#Converting datatype of appointment feature
actual_data.appointment=actual_data.appointment.astype("category")

In [10]:
actual_data.dtypes

summary             object
data                object
categories        category
sub_categories    category
appointment       category
dtype: object

In [28]:
actual_data.describe()

,summary,data,categories,sub_categories,appointment
count,57280,57280,57280,57280,57278
unique,41223,53680,8,22,5
top,nan,nan,PRESCRIPTION,MEDICATION RELATED,No
freq,3371,1746,15096,10598,41559


In [11]:
#listing all lables in categories column
unique_cat_labels = list(actual_data.categories.unique())
unique_cat_labels

['PRESCRIPTION',
 'ASK_A_DOCTOR',
 'MISCELLANEOUS',
 'APPOINTMENTS',
 'LAB',
 'JUNK',
 'mISCELLANEOUS',
 'asK_A_DOCTOR']

In [12]:
#listing all lables in categories column
unique_sub_cat_labels = list(actual_data.sub_categories.unique())
unique_sub_cat_labels

['REFILL',
 'MEDICATION RELATED',
 'OTHERS',
 'SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)',
 'QUERY ON CURRENT APPOINTMENT',
 'SYMPTOMS',
 'RESCHEDULING',
 'NEW APPOINTMENT',
 'PROVIDER',
 'SHARING OF LAB RECORDS (FAX ; E-MAIL ; ETC.)',
 'QUERIES FROM PHARMACY',
 'PRIOR AUTHORIZATION',
 'LAB RESULTS',
 'CANCELLATION',
 'CHANGE OF PROVIDER',
 'RUNNING LATE TO APPOINTMENT',
 'QUERIES FROM INSURANCE FIRM',
 'CHANGE OF HOSPITAL',
 'FOLLOW UP ON PREVIOUS REQUEST',
 'CHANGE OF PHARMACY',
 'JUNK',
 'mEDICATION RELATED']

In [13]:
#considering all lables in categories as Train_Y
train_y = np.array([unique_cat_labels.index(i) for i in actual_data.categories])
train_y 

array([0, 1, 1, ..., 2, 2, 2])

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

#tokenizing words in data column
data_tokenizer = Tokenizer(num_words=10000)
data_tokenizer.fit_on_texts(actual_data.data)

In [82]:
#tokenizing words in summary column
#summary_tokenizer = Tokenizer(num_words=10000)
#summary_tokenizer.fit_on_texts(actual_data.summary)

In [92]:
#converting to sequence
#train_summary_x = summary_tokenizer.texts_to_sequences(actual_data.summary)
#train_summary_x = pad_sequences(train_summary_x, maxlen=50)
#print(train_summary_x.shape)

In [15]:
#train_summary_x

In [16]:
#converting to sequence
train_data_x = data_tokenizer.texts_to_sequences(actual_data.data)
train_data_x = pad_sequences(train_data_x, maxlen=50)
print(train_data_x.shape)

(57280, 50)


In [17]:
train_data_x

array([[   2,  538,   44, ...,  174,  892,   30],
       [ 451,   13,  216, ...,  227,  104,   30],
       [  17,   15,  629, ...,    1,   53,   30],
       ...,
       [   0,    0,    0, ...,  225,   53,   30],
       [ 471, 5576,  715, ...,  574,  136,  136],
       [  41,    2,  146, ..., 1744,  506,  214]])

In [18]:
train_x=train_data_x

In [19]:
#train_x=np.concatenate([train_summary_x,train_data_x])
#train_x

In [20]:
#each vector is converting as categorical
train_y=to_categorical(train_y)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.20, random_state=42)

In [22]:
model = Sequential()
model.add(Embedding(100000, 128))
model.add(LSTM(200,return_sequences=True,dropout=0.2,recurrent_dropout=0.2))
model.add(LSTM(128,return_sequences=True))
model.add(LSTM(128,return_sequences=False))
model.add(Dense(8))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [23]:
X_train.shape

(45824, 50)

In [24]:
X_train

array([[   0,    0,    0, ...,   96,   96,   40],
       [   0,    0,    0, ...,  295,  104,   42],
       [ 159,  295,  104, ..., 2044,  468,   45],
       ...,
       [  25,    8,  914, ..., 1394,  154,  433],
       [ 341,   12,   66, ...,  341,   12,   66],
       [   1,  243,  235, ...,    1,  169,   93]])

In [25]:
#batch size limits the number of samples to be shown to the network before a weight update can be performed
batch_size=70
model.fit(X_train,
          y_train,
          batch_size=batch_size,
          epochs=8)

Epoch 1/5
45824/45824 [==============================] - 399s 9ms/step - loss: 0.9939 - acc: 0.6207
Epoch 2/5
45824/45824 [==============================] - 433s 9ms/step - loss: 0.7740 - acc: 0.7179
Epoch 3/5
45824/45824 [==============================] - 463s 10ms/step - loss: 0.7115 - acc: 0.7406
Epoch 4/5
45824/45824 [==============================] - 478s 10ms/step - loss: 0.6711 - acc: 0.7538
Epoch 5/5
45824/45824 [==============================] - 447s 10ms/step - loss: 0.6326 - acc: 0.7680


In [26]:
test_predictions = model.predict_classes(X_test)

In [72]:
list = []
list1=[]
for i in test_predictions:
    print(unique_labels[i])
    list.append(unique_labels[i])
for j in y_test:
    list1.append(unique_labels[j])

REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
REFILL
REFILL
MEDICATION RELATED
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
REFILL
REFILL
REFILL
REFILL
MEDICATION RELATED
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
OTHERS
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
MEDICATION RELATED
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
REFILL
REFILL
REFILL
REFILL
REFILL
OTHERS
OTHERS
REFILL
OTHERS
MEDICATION RELATED
OTHERS
MEDICATION RELATED
QUERY ON CURRENT APPOINTMENT
MEDICATION RELATED
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
MEDICATION RELATED
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
REFILL
OTHERS
REFILL
MEDICATION RELATED
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
QUERY ON CURRENT APPOINTMENT
OTHERS
OTHERS
REFILL
OTH

SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
REFILL
OTHERS
REFILL
OTHERS
QUERY ON CURRENT APPOINTMENT
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
OTHERS
REFILL
OTHERS
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
MEDICATION RELATED
MEDICATION RELATED
QUERY ON CURRENT APPOINTMENT
OTHERS
OTHERS
QUERY ON CURRENT APPOINTMENT
REFILL
REFILL
MEDICATION RELATED
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
QUERY ON CURRENT APPOINTMENT
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
REFILL
MEDICATION RELATED
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
MEDICATION RELATED
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
OTHERS
OTHERS
REFILL
OTHERS
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RE

REFILL
MEDICATION RELATED
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
OTHERS
MEDICATION RELATED
OTHERS
OTHERS
REFILL
REFILL
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
QUERY ON CURRENT APPOINTMENT
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
REFILL
QUERY ON CURRENT APPOINTMENT
REFILL
REFILL
MEDICATION RELATED
REFILL
MEDICATION RELATED
REFILL
MEDICATION RELATED
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
OTHERS
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
REFILL
REFILL
OTHERS
REFILL
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
REF

REFILL
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
MEDICATION RELATED
MEDICATION RELATED
OTHERS
MEDICATION RELATED
OTHERS
MEDICATION RELATED
REFILL
MEDICATION RELATED
OTHERS
REFILL
REFILL
REFILL
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
REFILL
MEDICATION RELATED
REFILL
MEDICATION RELATED
OTHERS
OTHERS
REFILL
REFILL
REFILL
OTHERS
MEDICATION RELATED
OTHERS
QUERY ON CURRENT APPOINTMENT
OTHERS
OTHERS
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
QUERY ON CURRENT APPOINTMENT
MEDICATION RELATED
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
OTHERS
QUERY ON CURRENT APPOINTMENT
REFILL
REFILL
OTHERS
OTHERS
OTHERS
MEDICATION RELATED
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
REFILL
REFILL
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
QUERY ON CURRENT APPOINTMENT
REFILL
REFILL
REFILL
OTHERS
OTHERS
REFILL
MEDI

OTHERS
REFILL
MEDICATION RELATED
OTHERS
REFILL
REFILL
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
REFILL
OTHERS
QUERY ON CURRENT APPOINTMENT
MEDICATION RELATED
REFILL
REFILL
OTHERS
MEDICATION RELATED
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
MEDICATION RELATED
MEDICATION RELATED
REFILL
MEDICATION RELATED
REFILL
MEDICATION RELATED
OTHERS
REFILL
REFILL
QUERY ON CURRENT APPOINTMENT
MEDICATION RELATED
MEDICATION RELATED
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
REFILL
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
MEDICATION RELATED
REFILL
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALT

OTHERS
REFILL
REFILL
REFILL
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
REFILL
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
MEDICATION RELATED
REFILL
REFILL
REFILL
MEDICATION RELATED
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
REFILL
OTHERS
MEDICATION RELATED
OTHERS
QUERY ON CURRENT APPOINTMENT
OTHERS
OTHERS
REFILL
OTHERS
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
OTHERS
REFILL
REFILL
MEDICATION RELATED
QUERY ON CURRENT APPOINTMENT
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
OTHERS
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
MEDICATION RELATED
REFILL
REFILL
SHARING OF HEALTH RECORDS (FAX ; E-MAIL ; ETC.)
REFILL
MEDICATION RELATED
OTHERS
REFILL
REFILL
MEDICATION RELATE

TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print (cm)